# Minicaml: Stack-safe Recursion

In questa esercitazione estenderemo l'interprete con la possibilità di definire **funzioni ricorsive sicure**, cioè che prevengano lo stack overflow. 

Prendiamo per esempio la funzione

    let rec g x =  
        1 * (g x) 
    in g 5

Questa funzione diverge sempre, perchè se facciamo una chiamata `g 5` essa chiamerà ricorsivamente `g 5` all'infinito. Anche la funzione 

    let rec f x = 
        if x mod 2 = 1 then f (x - 1)
        else f (x + 1)
    in ...
    
diverge sempre, perchè se facciamo una chiamata `f 3` essa chiamerà `f 2`, `f 3`, `f 2`, `f 3`...

Quando eseguiamo queste funzioni, è necessario fare tutte le chiamate ricorsive fino a raggiungere l'inevitabile stack overflow? In realtà, ci basta notare che, nella valutazione di `f 2`, c'è una chiamata a `f 3`, che è già stato chiamato in precedenza. In altre parole, sappiamo che una funzione ricorsiva (unaria) diverge se effettua una chiamata identica a una già fatta in precedenza.

L'esercizio di oggi prevede di definire funzioni ricorsive *sicure*, che tengano traccia dei valori sui quali la funzione è già stata chiamata. Se nell'eseguire una funzione ricorsiva `f` so che è già stata chiamata sui valori `v1, v2, v2, v4`, quando effettuo una chiamata a `f v1` posso restituire errore immediatamente, perchè tanto la funzione divergerà.

    let safe rec f x = 
        if x mod 2 = 1 then f (x - 1)
        else f (x + 1)
    in f 2  (*da errore "funzione ricorsiva ciclica" senza effettuare stack overflow*)
    
**NOTA:** Questo esercizio ci permetterà di riconoscere *alcune* funzioni che divergono, ma non tutte! Una funzione

    let rec f x = f (x + 1)
    
diverge sempre anche se non effettua mai chiamate identiche. D'altro canto, scoprire se una funzione diverge prima di eseguirla è impossibile...

## Esercizio 1: Sintassi

Questa è la definizione della sintassi astratta dell'interprete di Minicaml.
Estendere questa definizione con un nuovo costrutto, per dichiarare funzioni ricorsive sicure

    let safe rec f x =  
        1 + (f x) 
    in f 5


In [ ]:
(* Identificatori *)
type ide = string;;


(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp

## Esercizio 2: Valori 

Questa è la definizione di ambiente e di valori esprimibili (`evT`).

Oltre a booleani, interi e simili, fra i valori possibili ci sono i valori funzione, le **closure**. Una closure contiene tutti le componenti necessarie ad applicare una funzione (eccezion fatta per il valore in input, ovviamente)
- `Closure(x, body, env)` contiene una espressione `body` in cui può comparire una variabile libera `x`. Inoltre, contiene anche un ambiente `env`, l'ambiente di definizione della funzione, che ci dà informazioni sul valore di tutte le variabili che sono in `body`, tranne `x`. In pratica, una closure è una "espressione con un buco", e `x` è il nome di quel buco. Durante una applicazione sapremo il valore `v` di input, e possiamo quindi riempire il buco e valutare il body.
- `RecClosure(f, x, body, env)`contiene tutte le informazioni di una normale closure. Inoltre, contiene anche il nome della funzione ricorsiva `f` che stiamo definendo, visto che dentro `body` potremmo trovare una chiamata alla funzione `f` stessa. In pratica, una recclosure è una "espressione con due buchi", che chiamiamo `x` e `f`. Durante una applicazione potremo "riempire" `x` con `v` e `f` con la recClosure stessa.

Una funzione sicura deve tenere traccia di quali valori sono stati già forniti come input durante una serie di chiamate ricorsive. Estendere la definizione di `evT` con una nuova `SafeClosure` adatta allo scopo.

**Hint:** Se non sapete come proseguire, studiate la funzione di valutazione, e capite che informazioni serve mantenere in una `SafeClosure`

In [ ]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | UnBound
    
(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

## Type Checking 

In [ ]:
(* I tipi *)
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound

(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(n) -> TInt
    | Bool(b) -> TBool
    | String(s) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | UnBound -> TUnBound
    
(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    |TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )

## Eccezione in caso di errori durante l'esecuzione

In [ ]:
(* Errori a runtime *)
exception RuntimeError of string

## Operazioni primitive

In [ ]:
(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")
    
(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")

## Esercizio 3: Interprete

Questa è la definizione della funzione di valutazione. È possibile vedere che il "ciclo di vita" di una funzione ha due step:
- Quando viene dichiarata, con un `let rec` o con un `fun`, viene creata una closure e salvata nell'ambiente
- Quando viene usata, con una `apply`, si prende la closure e si "applica" al parametro attuale.
Nel caso di una funzione non ricorsiva, "applicare" vuol dire estendere l'ambiente (quello della closure) con un nuovo binding fra parametro formale (quello della closure) e parametro attuale (quello della `apply`).  Nel caso di una funzione ricorsiva, in aggiunta bisogna anche fare un binding fra il nome della funzione `f` e la closure stessa, perchè dentro il body ci potrebbe essere una variabile libera `f`.
 
Estendere `eval` per trattare le funzioni sicure. Per estendere Minicaml con costrutti funzionali, bisogna specificare la semantica dei due step di cui sopra
- Per la definizione, dare la semantica dell'espressione definita nell'esercizio 1
- Per l'applicazione, estendere il codice di `Apply(eF, eArg)` con il caso in cui `eF` valuta a una `SafeClosure`

**HINT:** usare `List.mem` per controllare se un elemento è in una lista!

In [ ]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )
    